## 🔧 Instalación de Dependencias


## 📦 Importar Librerías


In [ ]:
# Variables globales para el sistema de backup
backup_thread_running = False
backup_thread = None


## 📁 Configuración de Datos


## 🔥 Funciones de Backup Automático en Drive


## 📊 Monitoreo en Tiempo Real


## 🚀 Inicialización del Sistema de Backup


## 🛑 Early Stopping Implementado

### ✅ **Configuración Agregada**
- **Patience**: 5 épocas (configurable en `DeformableDETRConfig`)
- **Métrica monitoreada**: `coco/bbox_mAP` (mean Average Precision)
- **Modo**: `max` (busca maximizar la métrica)
- **Delta mínimo**: 0.001 (mejora mínima requerida)

### 🎯 **¿Cómo funciona?**
- El entrenamiento se detendrá automáticamente si la métrica `coco/bbox_mAP` no mejora durante **5 épocas consecutivas**
- Esto evita el sobreajuste y ahorra tiempo de entrenamiento
- El modelo se guardará automáticamente cuando se active el early stopping

### ⚙️ **Personalización**
Para cambiar el comportamiento, modifica en la clase `DeformableDETRConfig`:
```python
self.patience = 10  # Más paciencia (10 épocas)
self.patience = 3   # Menos paciencia (3 épocas)
```


## 🚀 Entrenamiento del Modelo


## 📊 Evaluación y Visualización

## 📊 Cálculo de Métricas de Clasificación


## 🚀 Instrucciones de Uso - V1 Optimizado

### 📋 **Antes de Ejecutar:**
1. **Montar Google Drive** (se hace automáticamente)
2. **Verificar que tienes Colab Pro/Pro+** para mejor rendimiento
3. **Configurar datos** en la estructura correcta

### 🔥 **Características V1:**
- **Backup automático**: Los checkpoints se guardan en Drive cada 5 épocas
- **Recuperación automática**: Si se interrumpe, reanuda desde el último checkpoint
- **Configuración optimizada**: Batch size 8, workers 8 para mayor velocidad
- **Monitoreo en tiempo real**: Usa `show_training_status()` para ver progreso

### 📊 **Comandos Útiles Durante el Entrenamiento:**
```python
# Ver estado del entrenamiento
show_training_status()

# Hacer backup manual
backup_to_drive()

# Verificar backups en Drive
monitor_training_progress()
```

### ⚠️ **Importante:**
- **NO cierres la pestaña** de Colab durante el entrenamiento
- **Los backups se hacen automáticamente** cada 5 épocas
- **Si se interrumpe**, simplemente ejecuta de nuevo y reanudará automáticamente
- **Los checkpoints se guardan en**: `/content/drive/MyDrive/aerial-wildlife-count-results/deformable_detr_v1/`

### 🎯 **Optimizaciones de Velocidad:**
- **Batch size**: 8 (vs 4 original)
- **Val batch size**: 4 (vs 2 original)
- **Workers**: 8 (vs 4 original)
- **Save period**: 1 época (vs 10 original)
- **Early stopping**: 5 épocas de paciencia


## 🔍 Inferencia y Pruebas


In [ ]:
# Cargar el mejor modelo entrenado
from mmdet.apis import init_detector, inference_detector
import mmcv

# Configurar el modelo para inferencia
config_file = f'{detr_config.work_dir}/config.py'
checkpoint_file = f'{detr_config.work_dir}/best_bbox_mAP_epoch_*.pth'

# Buscar el mejor checkpoint
import glob
checkpoint_files = glob.glob(f'{detr_config.work_dir}/best_bbox_mAP_epoch_*.pth')
if checkpoint_files:
    checkpoint_file = checkpoint_files[0]
    print(f"✅ Usando checkpoint: {checkpoint_file}")
else:
    print("❌ No se encontró el mejor checkpoint")
    checkpoint_file = f'{detr_config.work_dir}/latest.pth'

# Inicializar el modelo
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# Función para realizar inferencia en una imagen
def test_inference(image_path, score_threshold=0.3):
    """
    Realizar inferencia en una imagen
    
    Args:
        image_path: Ruta a la imagen
        score_threshold: Umbral de confianza para las detecciones
    
    Returns:
        result: Resultado de la detección
    """
    result = inference_detector(model, image_path)
    
    # Filtrar detecciones por umbral de confianza
    filtered_result = []
    for class_id, detections in enumerate(result):
        if len(detections) > 0:
            # detections: [x1, y1, x2, y2, score]
            filtered_detections = detections[detections[:, 4] >= score_threshold]
            filtered_result.append(filtered_detections)
        else:
            filtered_result.append(detections)
    
    return filtered_result

# Función para visualizar resultados
def visualize_inference(image_path, result, save_path=None):
    """
    Visualizar resultados de inferencia
    
    Args:
        image_path: Ruta a la imagen original
        result: Resultado de la detección
        save_path: Ruta para guardar la imagen con detecciones
    """
    import matplotlib.pyplot as plt
    import cv2
    
    # Cargar imagen
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Crear figura
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(image)
    
    # Colores para cada clase
    colors = ['red', 'blue', 'green', 'yellow', 'purple', 'orange']
    
    # Dibujar detecciones
    for class_id, detections in enumerate(result):
        if len(detections) > 0:
            color = colors[class_id % len(colors)]
            class_name = detr_config.classes[class_id]
            
            for detection in detections:
                x1, y1, x2, y2, score = detection
                
                # Dibujar bounding box
                rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                                   fill=False, color=color, linewidth=2)
                ax.add_patch(rect)
                
                # Agregar etiqueta
                label = f'{class_name}: {score:.2f}'
                ax.text(x1, y1-5, label, color=color, fontsize=10, 
                       bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))
    
    ax.set_title(f'Detecciones - {len([d for d in result if len(d) > 0])} objetos encontrados')
    ax.axis('off')
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✅ Imagen guardada en: {save_path}")
    
    plt.show()

# Probar inferencia en una imagen de ejemplo
test_image_path = "data/test/example.jpg"  # Cambiar por una imagen real

if os.path.exists(test_image_path):
    print("🔍 Realizando inferencia en imagen de prueba...")
    result = test_inference(test_image_path, score_threshold=0.3)
    visualize_inference(test_image_path, result, 
                       save_path=f"{detr_config.work_dir}/inference_example.jpg")
else:
    print("⚠️  No se encontró imagen de prueba. Usar una imagen del dataset de test.")


## 💾 Guardar y Exportar Modelo


In [ ]:
# Guardar el modelo final y exportar a diferentes formatos
import shutil
from pathlib import Path

def save_final_model():
    """Guardar el modelo final y crear copias de seguridad"""
    
    work_dir = Path(detr_config.work_dir)
    
    # Crear directorio de modelo final
    final_model_dir = work_dir / "final_model"
    final_model_dir.mkdir(exist_ok=True)
    
    # Copiar el mejor checkpoint
    best_checkpoint = None
    checkpoint_files = list(work_dir.glob("best_bbox_mAP_epoch_*.pth"))
    if checkpoint_files:
        best_checkpoint = max(checkpoint_files, key=lambda x: x.stat().st_mtime)
        shutil.copy2(best_checkpoint, final_model_dir / "best_model.pth")
        print(f"✅ Mejor modelo copiado: {best_checkpoint.name}")
    
    # Copiar la configuración
    config_file = work_dir / "config.py"
    if config_file.exists():
        shutil.copy2(config_file, final_model_dir / "config.py")
        print("✅ Configuración copiada")
    
    # Copiar métricas
    metrics_files = list(work_dir.glob("*metrics*.json")) + list(work_dir.glob("*metrics*.csv"))
    for metrics_file in metrics_files:
        shutil.copy2(metrics_file, final_model_dir / metrics_file.name)
        print(f"✅ Métricas copiadas: {metrics_file.name}")
    
    # Crear archivo de información del modelo
    model_info = {
        "model_name": "Deformable DETR V1",
        "classes": detr_config.classes,
        "num_classes": len(detr_config.classes),
        "best_checkpoint": best_checkpoint.name if best_checkpoint else "N/A",
        "training_epochs": detr_config.epochs,
        "batch_size": detr_config.batch_size,
        "optimizer": "AdamW",
        "backbone": "ResNet-50",
        "neck": "Deformable DETR",
        "head": "Deformable DETR",
        "created_date": str(datetime.now()),
        "description": "Modelo Deformable DETR entrenado para detección de vida silvestre aérea"
    }
    
    import json
    with open(final_model_dir / "model_info.json", 'w') as f:
        json.dump(model_info, f, indent=2)
    print("✅ Información del modelo guardada")
    
    return final_model_dir

def export_to_onnx():
    """Exportar modelo a formato ONNX"""
    try:
        from mmdeploy.apis import torch2onnx
        
        work_dir = Path(detr_config.work_dir)
        config_file = work_dir / "config.py"
        checkpoint_file = work_dir / "best_model.pth"
        
        if not checkpoint_file.exists():
            # Buscar el mejor checkpoint
            checkpoint_files = list(work_dir.glob("best_bbox_mAP_epoch_*.pth"))
            if checkpoint_files:
                checkpoint_file = max(checkpoint_files, key=lambda x: x.stat().st_mtime)
        
        if config_file.exists() and checkpoint_file.exists():
            # Crear directorio de exportación
            export_dir = work_dir / "onnx_export"
            export_dir.mkdir(exist_ok=True)
            
            # Exportar a ONNX
            torch2onnx(
                str(config_file),
                str(checkpoint_file),
                str(export_dir),
                input_shape=(1, 3, 640, 640),
                device='cuda:0'
            )
            print("✅ Modelo exportado a ONNX")
            return export_dir
        else:
            print("❌ No se encontraron archivos de configuración o checkpoint")
            return None
            
    except ImportError:
        print("⚠️  mmdeploy no está instalado. No se puede exportar a ONNX")
        return None
    except Exception as e:
        print(f"❌ Error al exportar a ONNX: {e}")
        return None

def create_model_package():
    """Crear paquete completo del modelo"""
    
    work_dir = Path(detr_config.work_dir)
    package_dir = work_dir / "model_package"
    package_dir.mkdir(exist_ok=True)
    
    # Guardar modelo final
    final_model_dir = save_final_model()
    
    # Copiar directorio final al paquete
    shutil.copytree(final_model_dir, package_dir / "model", dirs_exist_ok=True)
    
    # Crear README del modelo
    readme_content = f"""# Deformable DETR V1 - Modelo de Detección de Vida Silvestre Aérea

## Descripción
Modelo Deformable DETR entrenado para detectar animales en imágenes aéreas.

## Clases Detectadas
{chr(10).join([f"- {i}: {class_name}" for i, class_name in enumerate(detr_config.classes)])}

## Archivos Incluidos
- `best_model.pth`: Checkpoint del mejor modelo
- `config.py`: Configuración del modelo
- `model_info.json`: Información detallada del modelo
- `*metrics*.json`: Métricas de entrenamiento
- `*metrics*.csv`: Métricas en formato CSV

## Uso
```python
from mmdet.apis import init_detector, inference_detector

# Cargar modelo
config_file = 'config.py'
checkpoint_file = 'best_model.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# Realizar inferencia
result = inference_detector(model, 'image.jpg')
```

## Especificaciones Técnicas
- **Backbone**: ResNet-50
- **Neck**: Deformable DETR
- **Head**: Deformable DETR
- **Optimizador**: AdamW
- **Batch Size**: {detr_config.batch_size}
- **Épocas**: {detr_config.epochs}

## Fecha de Creación
{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""
    
    with open(package_dir / "README.md", 'w', encoding='utf-8') as f:
        f.write(readme_content)
    
    print("✅ Paquete del modelo creado")
    return package_dir

# Ejecutar funciones de guardado y exportación
print("💾 Guardando modelo final...")
final_model_dir = save_final_model()

print("📦 Creando paquete del modelo...")
package_dir = create_model_package()

print("🔄 Intentando exportar a ONNX...")
onnx_dir = export_to_onnx()

print(f"""
✅ Modelo guardado exitosamente!

📁 Directorios creados:
- Modelo final: {final_model_dir}
- Paquete completo: {package_dir}
- ONNX export: {onnx_dir if onnx_dir else 'No disponible'}

📋 Archivos principales:
- best_model.pth: Checkpoint del mejor modelo
- config.py: Configuración del modelo
- model_info.json: Información del modelo
- README.md: Documentación del modelo
""")


## 🎉 ¡Entrenamiento Completado!

### 📋 Resumen del Entrenamiento
- **Modelo**: Deformable DETR con backbone {detr_config.backbone}
- **Tamaño de imagen**: {detr_config.imgsz}x{detr_config.imgsz}
- **Épocas**: {detr_config.epochs}
- **Early Stopping**: {detr_config.patience} épocas de paciencia
- **Clases detectadas**: {len(detr_config.classes)} especies de animales

### 📊 Próximos Pasos
1. **Evaluar métricas**: Revisar mAP, precision, recall
2. **Ajustar hiperparámetros**: Si es necesario mejorar el rendimiento
3. **Exportar modelo**: Convertir a ONNX o TorchScript para deployment
4. **Probar en nuevas imágenes**: Validar en datos no vistos

### 🔧 Configuración Personalizada
Para modificar parámetros, edita la clase `DeformableDETRConfig` en la celda de configuración:
- Cambiar backbone: `"swin_t"` o `"resnet50"`
- Ajustar épocas: `epochs = 100`
- Modificar tamaño de imagen: `imgsz = 1024`
- Cambiar batch size: `batch_size = 8`
- Ajustar early stopping: `patience = 10` (más paciencia) o `patience = 3` (menos paciencia)

### 📚 Recursos Adicionales
- [Documentación MMDetection](https://mmdetection.readthedocs.io/)
- [Deformable DETR Paper](https://arxiv.org/abs/2010.04159)
- [DETR Paper](https://arxiv.org/abs/2005.12872)


## 💾 Guardar y Descargar Resultados